<a href="https://colab.research.google.com/github/HanbumKo/DRL-course/blob/main/0_gym/0_2_creating_gym_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Gym Env

이번 노트북에서는 gym 환경을 직접 제작하는 방법을 살펴보겠습니다.

환경을 제작하여서 python package 형태로 제공할 수도 있지만 jupyter notebook상에서는 gym 환경 클래스를 직접 정의하고 불러오도록 하겠습니다.

package 형태로 제작하는 방법은 [이곳](https://github.com/openai/gym/blob/master/docs/creating-environments.md)을 참고하시면 되겠습니다.



---





In [ ]:
import gym


class SimpleEnv(gym.Env):
    def __init__(self):
        # Some initialization for Env
        pass

    def step(self, action):
        obs, reward, done, info = 0, 0, True, {}
        return obs, reward, done, info

    def reset(self):
        obs = 0
        return obs

    def render(self):
        # Code for visualization
        pass

    def close(self):
        # Clear env
        pass

gym 환경을 구현하려면 gym.Env를 상속받는 클래스를 구현해야합니다.

위와같이 gym.Env를 상속받고 총 5개 함수를 쓰임새에 맞게 구현해주면 되겠습니다.

---



# GridWorld

![gridworld](https://github.com/HanbumKo/DRL-course/blob/main/0_gym/images/gridworld.png?raw=true)

위 그림과 같이 간단한 2x4 GridWorld 에서의 환경을 구현해보도록 하겠습니다. Agent(스마일)가 별을 찾도록 하는것을 목표로 하는 환경으로 구성해보겠습니다.


observation은 Agent위에 아무것도 없으면 0, 금지표시가 있으면 1, 별이 있으면 2로 표현하도록 하겠습니다.

action은 (위, 아래, 왼쪽, 오른쪽)로의 움직임을 각각 (0, 1, 2, 3)으로 사용하도록 하겠습니다.





In [ ]:
import gym
import numpy as np

from gym import spaces


class GridEnv(gym.Env):
    def __init__(self):
        self.world_shape = [2, 4]
        self.agent_pos = [1, 0]
        self.restrict_pos = [0, 1]
        self.star_pos = [0, 3]
        self.action_space = spaces.Discrete(4) # (위, 아래, 왼쪽, 오른쪽) 4가지 action
        self.obs_space = spaces.Discrete(3) # (빈 곳, 금지표시, 별) 3가지 observation

    def step(self, action):
        if action == 0:
            self.agent_pos[0] -= 1
        elif action == 1:
            self.agent_pos[0] += 1
        elif action == 2:
            self.agent_pos[1] -= 1
        elif action == 3:
            self.agent_pos[1] += 1
        else:
            raise Exception("the action is not defined")
    
        info = {}

        return self._get_obs(), self._get_reward(), self._is_done(), info

    def reset(self):
        self.world = np.zeros(self.world_shape)
        self.world[0, 1] = 1 # 금지표시
        self.world[0, 3] = 2 # 별
        self.agent_pos = [1, 0]

        return self._get_obs()

    def render(self):
        self.world[self.agent_pos[0], self.agent_pos[1]] = -1
        print(self.world)
        self.world[self.agent_pos[0], self.agent_pos[1]] = 0

    def close(self):
        pass

    def _get_obs(self):
        if self.agent_pos == [1, 1]:
            return 1 # 금지표시 아래
        elif self.agent_pos == [1, 3]:
            return 2 # 별 아래
        else:
            return 0 # 빈 곳
    
    def _get_reward(self):
        # 금지표시
        if self.agent_pos == self.restrict_pos:
            return -1
        # 별 cases
        elif self.agent_pos == self.star_pos:
            return 1
        # 빈곳
        else:
            return 0
    
    def _is_done(self):
        # 맵 밖으로 나갔을 때
        if self.agent_pos[0] < 0 or self.agent_pos[0] > 1:
            return True
        if self.agent_pos[1] < 0 or self.agent_pos[1] > 3:
            return True
        # 금지표시
        if self.agent_pos == self.restrict_pos:
            return True
        # 별
        if self.agent_pos == self.star_pos:
            return True
        # 나머지
        else:
            return False


위에서 설명한 GridWorld 환경을 코드로 구현한 결과입니다. action space를 4개의 discrete action으로 정의해 주었고 observation space는 3개의 discrete observation으로 설정해 주었습니다. 또한 


```
self.action_space = spaces.Discrete(4) # (위, 아래, 왼쪽, 오른쪽) 4가지 action
self.obs_space = spaces.Discrete(3) # (빈 곳, 금지표시, 별) 3가지 observation

```

step() 함수에서는 들어온 action에 따라 agent_pos를 조절 후 그에 맞게 다음 observation, reward, done을 반환 하도록 하였습니다.

render()에서는 간단히 numpy array를 print 해서 visualization을 해주는데, python의 GUI 라이브러리를 사용하여 표현해 줄 수도 있습니다.


In [ ]:
env = GridEnv()

for i_episode in range(20):
    obs = env.reset()
    for t in range(100):
        env.render()
        print(obs)
        # action = env.action_space.sample() # random action
        action = 3 # Move right
        obs, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

env.close()

이제 위에서 구현한 GridEnv를 사용해볼 차례입니다. action을 무조건 오른쪽(3)으로 주고 이에 따라 observation이 바뀌는 것을 확인해 볼 수 있습니다.

# Homework

새로운 GridWorld 환경을 만들어 16 x 16 크기로 구성한 다음 다음과 같이 구현해주세요.



1.   reset()시 agent의 위치, 별의 위치, 금지표시의 위치를 random하게 생성
2.   observation은 12개의 discrete space로 구성하여 1-4는 (위, 아래, 왼쪽, 오른쪽)으로 벽이 얼마나 떨어져 있는지, 5-8은 (위, 아래, 왼쪽, 오른쪽)으로 금지표시가 얼마나 떨어져 있는지, 9-12는 (위, 아래, 왼쪽, 오른쪽)으로 별 표시가 얼마나 떨어져 있는지 반환 (spaces.Tuple과 spaces.Discrete를 동시에 사용하여야 합니다. 사용방법은 [이곳](https://github.com/openai/gym-soccer/blob/master/gym_soccer/envs/soccer_env.py#L29-L34)을 참고해주세요.)
3.   reward는 예제와 같이 구성
4.   done도 예제와 같이 구성

## bonus

python의 GUI 라이브러리를 찾고 render function 사용 시 GUI로 볼 수 있도록 구성
